In [46]:
from typing import List, Tuple
from collections import defaultdict, deque

def find_shortest_paths(N: int, routes: List[Tuple[int, int, int]], start: int) -> List[int]:
    """
    Find shortest paths using SPFA (Shortest Path Faster Algorithm)
    """
    dist = [float('inf')] * (N + 1)
    dist[start] = 0
    
    # Create adjacency list
    graph = defaultdict(list)
    for u, v, w in routes:
        graph[u].append((v, w))
    
    # Initialize queue with start node
    queue = deque([start])
    in_queue = [False] * (N + 1)
    in_queue[start] = True
    
    # Count relaxations for each vertex to detect negative cycles
    relaxation_count = [0] * (N + 1)
    
    while queue:
        u = queue.popleft()
        in_queue[u] = False
        
        # Process all neighbors
        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                relaxation_count[v] += 1
                
                # Check for negative cycle
                if relaxation_count[v] >= N:
                    dist[v] = float('-inf')
                    continue
                
                # Add to queue if not already present
                if not in_queue[v]:
                    queue.append(v)
                    in_queue[v] = True
    
    return dist

def calculate_assignments(N: int, K: int, distances: List[int], people_needs: List[int], C: int) -> Tuple[List[int], List[Tuple[int, int, int]]]:
    """
    Calculate worker assignments to each country
    """
    # Create list of available countries with their distances
    available_countries = []
    for country in range(1, N + 1):
        if people_needs[country - 1] > 0 and distances[country] != float('inf'):
            dist = distances[country]
            available_countries.append((dist, country))
    
    # Sort by total cost (distance + C)
    available_countries.sort(key=lambda x: x[0] if x[0] != float('-inf') else float('-inf'))
    
    results = []
    assignments = []
    workers_assigned = 0
    
    # Assign workers to countries
    for dist, country in available_countries:
        while people_needs[country - 1] > 0 and workers_assigned < K:
            total_cost = dist + C if dist != float('-inf') else float('-inf')
            # Convert -inf to a very negative number for practical purposes
            if total_cost == float('-inf'):
                total_cost = "-inf"
            
            results.append(total_cost)
            assignments.append((workers_assigned + 1, country, total_cost))
            people_needs[country - 1] -= 1
            workers_assigned += 1
    
    # Fill remaining workers with -1
    while workers_assigned < K:
        results.append(-1)
        assignments.append((workers_assigned + 1, None, -1))
        workers_assigned += 1
    
    return results, assignments

def runtest(N: int, M: int, C: int, people_needs: List[int], routes: List[Tuple[int, int, int]], K: int):
    """
    Run test with direct data input
    """
    try:
        # Find shortest paths from country 1
        distances = find_shortest_paths(N, routes, 1)
        
        # Calculate assignments
        results, assignments = calculate_assignments(N, K, distances, people_needs.copy(), C)
        
        # Display Results
        print("\nResults:")
        print("-" * 50)
        print(f"{'Worker':<8} {'Country':<10} {'Total Cost':<15}")
        print("-" * 50)
        
        for person, country, cost in assignments:
            if cost == -1:
                print(f"{person}\t\t {'N/A'}\t {'-1':<15}")
            else:
                print(f"{person}\t\t {country}\t\t {cost}")
                
        # print("\nCosts:")
        # print(*results, sep='\n')
            
    except Exception as e:
        print(f"Error: {e}")

In [47]:
# 7.6
N, M, C = 8, 11, 100
people_needs = [1, 2, 1, 2, 2, 2, 1, 1]

routes = [
    (1, 2, 8),
    (1, 6, 4),
    (2, 3, 3),
    (2, 5, 5),
    (2, 8, 20),
    (4, 3, -10),
    (5, 4, 2),
    (5, 7, 12),
    (6, 7, -4),
    (8, 1, 6),
    (8, 3, 1)
]
K = 15
runtest(N, M, C, people_needs, routes, K)


Results:
--------------------------------------------------
Worker   Country    Total Cost     
--------------------------------------------------
1		 1		 100
2		 7		 100
3		 6		 104
4		 6		 104
5		 3		 105
6		 2		 108
7		 2		 108
8		 5		 113
9		 5		 113
10		 4		 115
11		 4		 115
12		 8		 128
13		 N/A	 -1             
14		 N/A	 -1             
15		 N/A	 -1             


In [48]:
# 7.7
N, M, C = 11, 17, 100
people_needs = [1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2]
routes = [
    [1, 2, -1],
    [1, 6, 4],
    [1, 11, 2],
    [2, 3, -2],
    [2, 4, 9],
    [3, 4, -1],
    [3, 7, 1],
    [4, 5, -2],
    [5, 6, -1],
    [6, 7, -2],
    [7, 8, -1],
    [8, 1, -2],
    [8, 3, 5],
    [8, 5, 10],
    [9, 10, -2],
    [10, 11, -1],
    [11, 9, -1],
]
K = 20

# Run the test
runtest(N, M, C, people_needs, routes, K)


Results:
--------------------------------------------------
Worker   Country    Total Cost     
--------------------------------------------------
1		 6		 -inf
2		 7		 -inf
3		 7		 -inf
4		 11		 -inf
5		 11		 -inf
6		 5		 48
7		 4		 50
8		 4		 50
9		 3		 51
10		 3		 51
11		 2		 53
12		 1		 54
13		 8		 56
14		 8		 56
15		 10		 75
16		 9		 77
17		 N/A	 -1             
18		 N/A	 -1             
19		 N/A	 -1             
20		 N/A	 -1             
